In [6]:
from pysat.formula import CNF
from pysat.solvers import Glucose3

class House:
    """
    Represents a house in the puzzle with its attributes.
    """
    # Constants for categories and their attributes
    NATIONALITY, COLOR, DRINK, PET, CIGARETTE = 0, 1, 2, 3, 4
    ENGLISHMAN, SPANIARD, UKRAINIAN, NORWEGIAN, JAPANESE = 0, 1, 2, 3, 4
    RED, GREEN, IVORY, YELLOW, BLUE = 0, 1, 2, 3, 4
    COFFEE, TEA, MILK, ORANGE_JUICE, WATER = 0, 1, 2, 3, 4
    DOG, SNAILS, ZEBRA, FOX, HORSE = 0, 1, 2, 3, 4
    OLD_GOLD, KOOLS, CHESTERFIELDS, LUCKY_STRIKE, PARLIAMENTS = 0, 1, 2, 3, 4

    def __init__(self, number):
        self.number = number

class EinsteinPuzzle:
    """
    Represents the Einstein's Riddle puzzle with its constraints and solution.
    """
    # Constants for the problem with five houses and five categories
    K = 5  # Number of houses
    CATEGORIES = 5  # Categories: Nationality, Color, Drink, Pet, Cigarette
    M = 5  # Number of attributes per category

    def __init__(self):
        self.houses = [House(i) for i in range(self.K)]
        self.formula = CNF()

    def var(self, house, category, attribute):
        return house * self.CATEGORIES * self.M + category * self.M + attribute + 1

    def add_constraints(self):
        # ... (basic constraints for uniqueness and house attributes) ...

        # The Englishman lives in the red house
        for house in range(self.K):
            self.formula.append([-self.var(house, House.NATIONALITY, House.ENGLISHMAN), self.var(house, House.COLOR, House.RED)])
            self.formula.append([-self.var(house, House.COLOR, House.RED), self.var(house, House.NATIONALITY, House.ENGLISHMAN)])

        # The Spaniard owns the dog
        for house in range(self.K):
            self.formula.append([-self.var(house, House.NATIONALITY, House.SPANIARD), self.var(house, House.PET, House.DOG)])
            self.formula.append([-self.var(house, House.PET, House.DOG), self.var(house, House.NATIONALITY, House.SPANIARD)])

        # Clue 3: Coffee is drunk in the green house
        for house in range(self.K):
            self.formula.append([-self.var(house, House.DRINK, House.COFFEE), self.var(house, House.COLOR, House.GREEN)])
            self.formula.append([-self.var(house, House.COLOR, House.GREEN), self.var(house, House.DRINK, House.COFFEE)])

        # Clue 4: The Ukrainian drinks tea
        for house in range(self.K):
            self.formula.append([-self.var(house, House.NATIONALITY, House.UKRAINIAN), self.var(house, House.DRINK, House.TEA)])
            self.formula.append([-self.var(house, House.DRINK, House.TEA), self.var(house, House.NATIONALITY, House.UKRAINIAN)])

        # Clue 5: The green house is immediately to the right of the ivory house
        for house in range(self.K - 1):  # -1 because the last house can't have a house to its right
            self.formula.append([-self.var(house, House.COLOR, House.IVORY), self.var(house + 1, House.COLOR, House.GREEN)])
        
        # Clue 6: The Old Gold smoker owns snails
        for house in range(self.K):
            self.formula.append([-self.var(house, House.CIGARETTE, House.OLD_GOLD), self.var(house, House.PET, House.SNAILS)])
            self.formula.append([-self.var(house, House.PET, House.SNAILS), self.var(house, House.CIGARETTE, House.OLD_GOLD)])
    
        # Clue 7: Kools are smoked in the yellow house
        for house in range(self.K):
            self.formula.append([-self.var(house, House.CIGARETTE, House.KOOLS), self.var(house, House.COLOR, House.YELLOW)])
            self.formula.append([-self.var(house, House.COLOR, House.YELLOW), self.var(house, House.CIGARETTE, House.KOOLS)])
    
        # Clue 8: Milk is drunk in the middle house
        self.formula.append([self.var(2, House.DRINK, House.MILK)])
    
        # Clue 9: The Norwegian lives in the first house
        self.formula.append([self.var(0, House.NATIONALITY, House.NORWEGIAN)])
    
        # Clue 10: The man who smokes Chesterfields lives in the house next to the man with the fox
        for house in range(1, self.K - 1):  # Excluding the first and the last houses for simplicity, but they can be included with extra conditions
            self.formula.append([-self.var(house, House.CIGARETTE, House.CHESTERFIELDS), self.var(house+1, House.PET, House.FOX)])
            self.formula.append([-self.var(house, House.CIGARETTE, House.CHESTERFIELDS), self.var(house-1, House.PET, House.FOX)])
    
        # Clue 11: Kools are smoked in the house next to the house where the horse is kept
        for house in range(1, self.K - 1):  # Similarly excluding the first and last houses
            self.formula.append([-self.var(house, House.CIGARETTE, House.KOOLS), self.var(house+1, House.PET, House.HORSE)])
            self.formula.append([-self.var(house, House.CIGARETTE, House.KOOLS), self.var(house-1, House.PET, House.HORSE)])
    
        # Clue 12: The Lucky Strike smoker drinks orange juice
        for house in range(self.K):
            self.formula.append([-self.var(house, House.CIGARETTE, House.LUCKY_STRIKE), self.var(house, House.DRINK, House.ORANGE_JUICE)])
            self.formula.append([-self.var(house, House.DRINK, House.ORANGE_JUICE), self.var(house, House.CIGARETTE, House.LUCKY_STRIKE)])
    
        # Clue 13: The Japanese smokes Parliaments
        for house in range(self.K):
            self.formula.append([-self.var(house, House.NATIONALITY, House.JAPANESE), self.var(house, House.CIGARETTE, House.PARLIAMENTS)])
            self.formula.append([-self.var(house, House.CIGARETTE, House.PARLIAMENTS), self.var(house, House.NATIONALITY, House.JAPANESE)])
    
        # Clue 14: The Norwegian lives next to the blue house
        for house in range(1, self.K - 1):  # Similarly excluding the first and last houses
            self.formula.append([-self.var(house, House.NATIONALITY, House.NORWEGIAN), self.var(house+1, House.COLOR, House.BLUE)])
            self.formula.append([-self.var(house, House.NATIONALITY, House.NORWEGIAN), self.var(house-1, House.COLOR, House.BLUE)])
    def solve(self):
        solver = Glucose3()
        solver.append_formula(self.formula)
        return solver.solve()
        

if __name__ == "__main__":
    # Create an instance of the puzzle and add constraints
    puzzle = EinsteinPuzzle()
    puzzle.add_constraints()

    # Solve the puzzle
    solution = puzzle.solve()
    print(solution)


True
